## Import the Supporting Modules for this Notebook


In [1]:
import os
from random import choice
from uuid import uuid4
from datetime import datetime

from dotenv import load_dotenv
from IPython.display import Markdown, display
from requests import get

## Import LangChain Modules


In [2]:
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_mistralai import ChatMistralAI

### Langchain Globals


In [3]:
from langchain.globals import set_debug, set_verbose
from langchain.callbacks.tracers import ConsoleCallbackHandler

## Load Keys

Be sure you create the appropiate .env file in your folder. This includes your Langsmith Keys.


In [4]:
load_dotenv()

anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY")
mistral_api_key = os.getenv("MISTRAL_API_KEY")
openweather_api_key = os.getenv("OPENWEATHER_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

## Template Creation


### Base Template


In [5]:
template = "You are a very friendly and helpful Assistant. Please help the user with their queries. User: {query}"
prompt = ChatPromptTemplate.from_template(template=template)

### Pirate Template


In [6]:
pirate_template = "Avast, me hearty! Ye be a friendly and helpful Assistant. Translate this sentence into Pirate, the language of the high seas. I be askin' ye to translate this here passage fer me, if ye please. {passage}"
pirate_prompt = ChatPromptTemplate.from_template(template=pirate_template)

## LLM creation

In this example we are using Google Gemini and Anthropics Claude 2.1


In [7]:
gemini_llm = ChatGoogleGenerativeAI(google_api_key=google_api_key, model="gemini-pro")
anthropic_llm = ChatAnthropic(api_key=anthropic_api_key, model="claude-3-opus-20240229")
mistral_llm = ChatMistralAI(model="mistral-large-latest")

## LCEL Creation

### Basic Query

Here we'll create a few basic invokes and chains for our demonstration.


In [8]:
# Basic query

basic_query_g = prompt | gemini_llm | StrOutputParser()
basic_query_a = prompt | anthropic_llm | StrOutputParser()
basic_query_m = prompt | mistral_llm | StrOutputParser()

### Chaining

Next let's create a simple chain


In [9]:
# Simple chain that translates an LLM output into pirate

# User's query -> Google LLM -> Google LLM output -> Anthropic LLM -> Anthropic LLM output in Pirate
anthropic_translates = pirate_prompt | anthropic_llm
pirate_query = {"passage": basic_query_g} | anthropic_translates | StrOutputParser()

### API chain

To make it a little more interesting we can also try an API chain. First let's create a simple function that makes an api call to Openweathermap.org


In [10]:
# A simple API call to get the current weather


def get_weather(weather_api):
    api_data = weather_api.get("weather_api")
    weather_api_response = get(
        f"https://api.openweathermap.org/data/3.0/onecall?lat={api_data.lat}&lon={api_data.lon}&units={api_data.units}&appid={openweather_api_key}"
    )
    weather_api_response.raise_for_status()
    dt = datetime.fromtimestamp(weather_api_response.json()["current"]["dt"])
    formatted_time = (
        f'{dt.strftime("%I:%M %p")} Timezone={weather_api_response.json()["timezone"]}'
    )
    return {
        "local time": formatted_time,
        "curent weather": weather_api_response.json()["current"],
    }

Above you will notice I'm using the weather_api object in the function above. I will use Pydantic with Langchain. This allows helps ensure that the corret data is given and in a format that is usesable for my function. If I wanted to go further I could have added validation tests to ensure the data is valid for the api.


#### Build the Schema

Here I'd like to take a brief detour here from LCEL and LangSmith and show something that is important as we start working with structured output. LangChain helps us implement a Pydantic Model for our structured output.


In [11]:
# First create the model for our output parser
from typing import Literal

from langchain_core.pydantic_v1 import BaseModel, Field

Note that the docstrings here are crucial, as they will be passed along to the model along with the class name.


In [12]:
class WeatherLookup(BaseModel):
    """Multiply two integers together."""

    lat: float = Field(
        ...,
        title="Latitude",
        description="This is the latitude of the location you want to look up the weather for",
    )
    lon: float = Field(
        ...,
        title="Longitude",
        description="This is the longitude of the location you want to look up the weather for",
    )
    units: Literal["imperial", "metric"] = Field(
        "imperial",
        title="Unit",
        description="This is the unit you want the temperature to be in (imperial for F or metric for C)",
    )

We will create a prompt for the user to make a query in natural language to ask for the weather in any desired location. The docsstrings in the Pydantic Model tell the LLM how the output should be structured. While it is possible to just provide the query without instruction, by providing instructions, the LLM can provide more accurate results and more focused responses.


In [13]:
# This is the template for the LLM to understand our request.
weather_template = "You are a very friendly and helpful Assistant. Please help the user with their weather related queries. User: {query}"
weather_prompt = ChatPromptTemplate.from_template(template=weather_template)

# We will create a weather man prompt for the generative ouptut at the end of the chain

weather_man_template = "This is the response from api.openweather.com about the current weather conditions. Based on this response, can you please create a friendly weatherman report and include relevant emojis that are fun and personalized to me? Also, please provide the current time, which is provided in the API output you have received as well.\n\n{weather_api_response}"
weather_man_prompt = ChatPromptTemplate.from_template(template=weather_man_template)

#### Configure the LLM and the Chain

There are couple things going on here:


In [14]:
# We will use Mistral for the first step, for the structure of the output
from langchain_core.runnables import RunnableLambda


structured_llm_m = mistral_llm.with_structured_output(WeatherLookup)

retrieve_location_chain = {
    "weather_api": (weather_prompt | structured_llm_m)
} | RunnableLambda(get_weather)

weather_output = weather_man_prompt | gemini_llm | StrOutputParser()

api_chain = {"weather_api_response": retrieve_location_chain} | weather_output

/Users/barweiss/.pyenv/versions/3.11.8/envs/langsmith-py3.11.8/lib/python3.11/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `with_structured_output` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


## LangSmith

First let's run the chains and then review them in LangSmith.

To use the out of the box functionality you just need to have the following in your environment:

```ini
# Langchain Settings
LANGCHAIN_TRACING_V2=true
LANGCHAIN_ENDPOINT=https://api.smith.langchain.com
LANGCHAIN_API_KEY=<API_KEY>
LANGCHAIN_PROJECT=barry-local-dev # this is the project name the traces will be found in LangSmith
```


### Run the basic query


In [15]:
basic_query_g.invoke(
    {
        "query": "Can you create a social media post for my blog about Artificial Intelligence and its impacts on our daily lives?"
    }
)

"Assistant: Of course! Here is a draft social media post for your blog about Artificial Intelligence and its impacts on our daily lives:\n\n**Headline:** Artificial Intelligence: Shaping Our Daily Lives\n\n**Body:**\n\nIn today's rapidly evolving technological landscape, Artificial Intelligence (AI) is making significant waves and reshaping various aspects of our daily lives. From our smartphones to healthcare, AI is transforming the way we interact with the world around us.\n\nIn this blog post, we delve into the fascinating world of AI, exploring its profound impacts on our daily routines. We'll discuss how AI is revolutionizing industries, improving our health, and even enhancing our creativity.\n\nJoin us as we uncover the transformative power of AI and its implications for our future.\n\n**Call to action:**\n\nVisit our blog today to read the full article and learn more about the exciting world of AI. Share your thoughts and experiences with us in the comments below!\n\n**Hashtags

In [16]:
basic_query_a.invoke(
    {
        "query": "Can you create a social media post for my blog about Artificial Intelligence and its impacts on our daily lives?"
    }
)

'Of course! Here\'s a social media post you can use to promote your blog about Artificial Intelligence and its impacts on our daily lives:\n\nTitle: "The AI Revolution: How Artificial Intelligence is Transforming Our World"\n\nPost:\n"Artificial Intelligence (AI) is no longer a thing of the future – it\'s here, and it\'s changing the way we live, work, and interact with the world around us. From virtual assistants like Siri and Alexa to self-driving cars and personalized recommendations on Netflix, AI is becoming an integral part of our daily lives.\n\nBut what does this mean for us? In my latest blog post, I explore the profound impacts of AI on various aspects of our lives, including:\n\n- How AI is revolutionizing industries such as healthcare, finance, and education\n- The potential benefits and risks of AI, from increased efficiency to job displacement\n- The ethical considerations surrounding AI development and deployment\n- How we can prepare ourselves for a future increasingly 

In [17]:
basic_query_m.invoke(
    {
        "query": "Can you create a social media post for my blog about Artificial Intelligence and its impacts on our daily lives?"
    }
)

"Absolutely, I'd be happy to help! Here's a suggestion for your social media post:\n\n🚀New Blog Post Alert!🚀\nDive into the fascinating world of Artificial Intelligence (AI) and discover how it's subtly, yet profoundly, reshaping our daily lives. From voice assistants that understand our commands to tailored recommendations that make our online experiences more personal, AI is everywhere! 🌐🤖\nJoin us as we explore the ins and outs of AI, its current impacts, and what the future might hold. Don't forget to share your thoughts on how AI has influenced your life too! 💭\nClick the link in our bio to read more. #ArtificialIntelligence #AI #DailyLife #TechBlog #Innovation"

### Run the simple chain

Let's run a simple chain.


In [18]:
# So that we can see the conversation in realtime let's turn on verbose mode
set_verbose(True)

print(
    pirate_query.invoke(
        {
            "query": "In regard to George Lucas's Space opera, Empire Strikes Back, can you recite the dialogue between Darth Vadar and Luke Skywalker during their duel on Bespin when Darth Vadar reveals to Luke that he is Luke's Father? Please provide the dialoge only."
        },
        config={"callbacks": [ConsoleCallbackHandler()]},
    )
)

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "query": "In regard to George Lucas's Space opera, Empire Strikes Back, can you recite the dialogue between Darth Vadar and Luke Skywalker during their duel on Bespin when Darth Vadar reveals to Luke that he is Luke's Father? Please provide the dialoge only."
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<passage>] Entering Chain run with input:
{
  "query": "In regard to George Lucas's Space opera, Empire Strikes Back, can you recite the dialogue between Darth Vadar and Luke Skywalker during their duel on Bespin when Darth Vadar reveals to Luke that he is Luke's Father? Please provide the dialoge only."
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<passage> > 3:chain:RunnableSequence] Entering Chain run with input:
{
  "query": "In regard to George Lucas's Space opera, Empire Strikes Back, can you recite the dialogue between Darth Vadar and Luke Skywalker during the

[llm/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel<passage> > 3:chain:RunnableSequence > 5:llm:ChatGoogleGenerativeAI] [3.46s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "**Darth Vader:** Obi-Wan never told you what happened to your father.\n\n**Luke Skywalker:** He told me enough! He told me you killed him!\n\n**Darth Vader:** No. I am your father.\n\n**Luke Skywalker:** No. No. That's not true! That's impossible!\n\n**Darth Vader:** Search your feelings. You know it to be true.\n\n**Luke Skywalker:** No! No! No!\n\n**Darth Vader:** Luke, you can destroy the Emperor. He has foreseen this. It is your destiny. Join me, and together we can rule the galaxy as father and son.\n\n**Luke Skywalker:** I'll never join you!\n\n**Darth Vader:** If you only knew the power of the dark side. Obi-Wan never told you what happened to your father.\n\n**Luke Skywalker:** He told me enough! He told me you killed him!\n\n**Darth Vader:** No. I am your fathe

### Run the API Chain


In [19]:
print(
    api_chain.invoke(
        {
            "query": "What is the weather like at Disney World today in Orlando, Florida?"
        },
        config={"callbacks": [ConsoleCallbackHandler()]},
    )
)

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "query": "What is the weather like at Disney World today in Orlando, Florida?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<weather_api_response>] Entering Chain run with input:
{
  "query": "What is the weather like at Disney World today in Orlando, Florida?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<weather_api_response> > 3:chain:RunnableSequence] Entering Chain run with input:
{
  "query": "What is the weather like at Disney World today in Orlando, Florida?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<weather_api_response> > 3:chain:RunnableSequence > 4:chain:RunnableParallel<weather_api>] Entering Chain run with input:
{
  "query": "What is the weather like at Disney World today in Orlando, Florida?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<weather_api_response> > 3:chain:RunnableSequence > 4:chain:Runna

[llm/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel<weather_api_response> > 3:chain:RunnableSequence > 4:chain:RunnableParallel<weather_api> > 5:chain:RunnableSequence > 7:llm:ChatMistralAI] [2.00s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "",
        "generation_info": {
          "finish_reason": "tool_calls"
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "",
            "additional_kwargs": {
              "tool_calls": [
                {
                  "function": {
                    "name": "WeatherLookup",
                    "arguments": "{\"lat\": 28.3852, \"lon\": -81.5639, \"units\": \"imperial\"}"
                  }
                }
              ]
            },
            "tool

## Configuring Tagging and Metadata


### Tagging

Let's turn off verbosity for now.


In [20]:
set_verbose(False)  # Turn off verbose mode

One whay of adding tags and metadata to a chain is the `with_config()` method of the `Runnable` class.


In [21]:
# Tagging the chains

# We are using the keyword "run_name" to tag the run

basic_query_g = basic_query_g.with_config({"run_name": "Basic Query with Google LLM"})
basic_query_a = basic_query_a.with_config(
    {"run_name": "Basic Query with Anthropic LLM"}
)
basic_query_m = basic_query_m.with_config({"run_name": "Basic Query with Mistral LLM"})

google_output = basic_query_g.invoke(
    {
        "query": "Can you create a social media post for my blog about Artificial Intelligence and its impacts on our daily lives?"
    }
)
anthropic_output = basic_query_a.invoke(
    {
        "query": "Can you create a social media post for my blog about Artificial Intelligence and its impacts on our daily lives?"
    }
)
mistral_output = basic_query_m.invoke(
    {
        "query": "Can you create a social media post for my blog about Artificial Intelligence and its impacts on our daily lives?"
    }
)
display(
    Markdown(
        f"""
                 **Google Output**
                 
                 {google_output}
                 
                 **Anthropic Output**
                 
                 {anthropic_output}
                 
                 **Mistral Output**
                 
                 {mistral_output}
                 """
    )
)


                 **Google Output**
                 
                 **Headline:** The Future is Here: AI's Profound Impacts on Our Daily Lives

**Body:**

Dive into the fascinating world of Artificial Intelligence (AI), a rapidly evolving technology that's transforming our lives in remarkable ways. From self-driving cars to personalized medicine, AI is leaving its mark on countless aspects of our daily routine.

In this blog post, we explore the profound impacts of AI on our:

* **Work:** Automating tasks, enhancing productivity, and creating new job opportunities
* **Health:** Improving diagnoses, developing innovative treatments, and promoting personalized healthcare
* **Education:** Providing tailored learning experiences, enhancing accessibility, and fostering collaboration
* **Transportation:** Revolutionizing mobility, reducing accidents, and optimizing traffic flow
* **Entertainment:** Enhancing creativity, personalizing experiences, and connecting us with stories and art

Join us as we delve into the exciting possibilities and potential challenges of AI. Discover how this technology is shaping our present and paving the way for an even more connected and automated future.

**#ArtificialIntelligence #AIImpacts #FutureTechnology #SmartLiving**
                 
                 **Anthropic Output**
                 
                 Certainly! Here's a sample social media post for your blog about Artificial Intelligence and its impacts on our daily lives:

Title: "The Impact of Artificial Intelligence on Our Everyday Lives"

🤖 Artificial Intelligence (AI) is transforming the way we live, work, and interact with the world around us. From virtual assistants like Siri and Alexa to personalized recommendations on streaming platforms, AI is becoming increasingly integrated into our daily routines.

In my latest blog post, I explore the various ways AI is shaping our lives, including:

✅ Improved efficiency and automation in industries such as healthcare, finance, and manufacturing
✅ Enhanced personalization and user experiences in e-commerce and entertainment
✅ Advancements in autonomous vehicles and smart home technologies
✅ Ethical considerations and the importance of responsible AI development

🔗 Read the full article to discover how AI is revolutionizing our world and what the future may hold as this technology continues to evolve. [Insert link to your blog post]

🤔 What are your thoughts on the impact of AI on our daily lives? Share your experiences and opinions in the comments below!

#ArtificialIntelligence #AI #Technology #Innovation #FutureOfAI #SmartTechnology #BlogPost

Feel free to customize this post to fit your blog's tone and style. Don't forget to include visuals such as images or videos to make your post more engaging and shareable on social media platforms.
                 
                 **Mistral Output**
                 
                 Absolutely, I'd be happy to help you craft a social media post for your blog. Here's a suggestion:

"🤖🌐 Dive into the fascinating world of Artificial Intelligence and discover how it's reshaping our daily lives in ways you might not even realize! From voice assistants to personalized recommendations, AI is everywhere. Check out our latest blog post to learn more about the impacts of AI and how it's transforming the future. 🚀💡 Link in bio #ArtificialIntelligence #AI #FutureTech #TechBlog"
                 

In [22]:
# Let's tag the remaining chains
priate_query = pirate_query.with_config({"run_name": "Pirate Translate Chain"})
api_chain = api_chain.with_config({"run_name": "Weather API Chain"})

In [24]:
priate_query.invoke(
    {
        "query": "In regard to George Lucas's Space opera, Empire Strikes Back, can you recite the dialogue between Darth Vadar and Luke Skywalker during their duel on Bespin when Darth Vadar reveals to Luke that he is Luke's Father? Please provide the dialoge only."
    }
)

"Ahoy, matey! I be happy t' help ye out with yer request. Here be me best attempt at translatin' that passage into th' tongue o' pirates:\n\n**Darth Vader:** Luke, me bucko... ye don't yet know how important ye be. Ye've only started t' discover yer true power. Join me, 'n together we can rule th' galaxy as father 'n son, savvy? \n\n**Luke Skywalker:** I'll never join ye, ye scurvy bilge rat!\n\n**Darth Vader:** If only ye knew th' power o' th' dark side, ye landlubber. Obi-Wan never told ye what happened t' yer dear ol' dad, did he?\n\n**Luke Skywalker:** He told me plenty, ye blackguard! Said ye killed 'im, he did!\n\n**Darth Vader:** Nay, me bucko. I be yer father, savvy?\n\n**Luke Skywalker:** Nay. Nay! 'Tis nothin' but lies! Impossible, says I!\n\n**Darth Vader:** Search yer feelings, ye sprog. Ye know it be true, deep in yer bones.\n\nI tried me best t' capture th' spirit 'o pirate speak while preservin' th' essence 'o th' original dialogue, me hearty. I hope this here translatio

In [25]:
api_chain.invoke({"query": "Whats the weather like on Chicago in Farenheit?"})

"**Personalized Weather Report**\n\nHey there, sunshine! ☀️\n\nIt's currently 10:25 PM in Chicago, and the weather is...\n\n🌡 **Temperature:** 53.33°F\n🌤 **Conditions:** Light drizzle 🌧️\n💨 **Wind:** 18.01 mph 💨\n💧 **Humidity:** 64%\n☁️ **Clouds:** 100% coverage\n\nLooks like you might want to grab an umbrella or raincoat before heading out! But don't worry, this drizzle will pass soon enough. 🤞\n\nEnjoy the rest of your evening, and stay dry! ☔"

### Metadata

First the class RunnableConfig will need to be imported.


In [27]:
from langchain_core.runnables import RunnableConfig

For example, if our application were tracking the user and the session id we can add that as meta data to the conversation. RunnableConfig also can be used to update the the following:

```python
"""Configuration for a Runnable."""

tags: List[str]
"""
Tags for this call and any sub-calls (eg. a Chain calling an LLM).
You can use these to filter calls.
"""

metadata: Dict[str, Any]
"""
Metadata for this call and any sub-calls (eg. a Chain calling an LLM).
Keys should be strings, values should be JSON-serializable.
"""

callbacks: Callbacks
"""
Callbacks for this call and any sub-calls (eg. a Chain calling an LLM).
Tags are passed to all callbacks, metadata is passed to handle*Start callbacks.
"""

run_name: str
"""
Name for the tracer run for this call. Defaults to the name of the class.
"""

max_concurrency: Optional[int]
"""
Maximum number of parallel calls to make. If not provided, defaults to
ThreadPoolExecutor's default.
"""

recursion_limit: int
"""
Maximum number of times a call can recurse. If not provided, defaults to 25.
"""

configurable: Dict[str, Any]
"""
Runtime values for attributes previously made configurable on this Runnable,
or sub-Runnables, through .configurable_fields() or .configurable_alternatives().
Check .output_schema() for a description of the attributes that have been made
configurable.
"""

run_id: Optional[uuid.UUID]
"""
Unique identifier for the tracer run for this call. If not provided, a new UUID
    will be generated.
"""
```


In [28]:
username = choice(["barweiss@cisco.com", "user1@cisco.com", "user2@cisco.com"])
session_id = str(uuid4())
google_output = basic_query_g.invoke(
    {
        "query": "Can you create a social media post for my blog about Artificial Intelligence and its impacts on our daily lives?"
    },
    RunnableConfig(metadata={"username": username, "session_id": session_id}),
)

username = choice(["barweiss@cisco.com", "user1@cisco.com", "user2@cisco.com"])
session_id = str(uuid4())
anthropic_output = basic_query_a.invoke(
    {
        "query": "Can you create a social media post for my blog about Artificial Intelligence and its impacts on our daily lives?"
    },
    RunnableConfig(metadata={"username": username, "session_id": session_id}),
)

username = choice(["barweiss@cisco.com", "user1@cisco.com", "user2@cisco.com"])
session_id = str(uuid4())
mistral_output = basic_query_m.invoke(
    {
        "query": "Can you create a social media post for my blog about Artificial Intelligence and its impacts on our daily lives?"
    },
    RunnableConfig(metadata={"username": username, "session_id": session_id}),
)
display(
    Markdown(
        f"""
                 **Google Output**
                 
                 {google_output}
                 
                 **Anthropic Output**
                 
                 {anthropic_output}
                 
                 **Mistral Output**
                 
                 {mistral_output}
                 """
    )
)


                 **Google Output**
                 
                 **Headline:** Unlocking the Power of AI: How It's Shaping Our Everyday World

**Body:**

In the realm of technology, Artificial Intelligence (AI) has emerged as a transformative force, seeping into countless aspects of our daily lives. From our smartphones to our workplaces, AI is quietly revolutionizing the way we interact with the world around us.

In this blog post, we dive into the fascinating world of AI and explore its profound impact on our society. We'll discuss how:

* **AI is enhancing our communication:** Chatbots and virtual assistants are streamlining communication, making it faster and more convenient.
* **AI is revolutionizing healthcare:** AI-powered diagnostics and treatments are improving patient outcomes and reducing healthcare costs.
* **AI is transforming transportation:** Self-driving cars and ride-sharing apps are revolutionizing the way we get around.
* **AI is boosting productivity:** AI-powered automation and decision support systems are freeing up our time and enhancing our efficiency.

But with great power comes great responsibility. It's important to consider the ethical implications of AI and ensure its responsible development and use.

Join us for an in-depth exploration of AI's transformative impact on our lives. Visit our blog today to learn more!

**Hashtags:** #ArtificialIntelligence #AI #Technology #Innovation #DailyLife
                 
                 **Anthropic Output**
                 
                 Certainly! Here's a social media post that you can use for your blog about Artificial Intelligence and its impacts on our daily lives:

"Artificial Intelligence: Transforming Our Daily Lives 🤖📱

From virtual assistants to personalized recommendations, AI is revolutionizing the way we live and interact with technology. 🌐 In my latest blog post, I dive into the fascinating world of AI and explore its profound impacts on our daily lives. 🔍

Discover how AI is shaping industries like healthcare, education, and transportation, and learn about the potential benefits and challenges that come with this groundbreaking technology. 🩺🎓🚗

🔗 Click the link below to read the full article and join the conversation! #AI #ArtificialIntelligence #Technology #Innovation #FutureOfTech"

Feel free to customize this post according to your blog's tone and style. Remember to include an eye-catching image or graphic related to AI to grab your audience's attention on social media. Don't forget to add relevant hashtags and a link to your blog post to drive traffic to your site.
                 
                 **Mistral Output**
                 
                 Absolutely, I'd be happy to help you craft a social media post for your blog. Here's a suggestion:

🛠️🧠 "Dive into the fascinating world of Artificial Intelligence and discover how it's subtly reshaping our everyday lives! From voice assistants to personalized recommendations, AI is no longer a thing of the future, but very much a part of our present. Join us on our blog as we explore the transformative impacts of AI and how it's making waves in various sectors. Let's embrace the change and understand it better, together. 🚀🌐

Link to the blog: [Your Blog Link]

#ArtificialIntelligence #AI #DailyLife #TechBlog #Innovation"
                 

Tagging Prompts, Output Parsers, etc
More than just LLM and Run can be tagged.


In [29]:
username = choice(["barweiss@cisco.com", "user1@cisco.com", "user2@cisco.com"])
session_id = str(uuid4())

template = "You are a very friendly and helpful Assistant. Please help the user with their queries. User: {query}"
prompt = ChatPromptTemplate.from_template(template=template)
tagging_test = (
    prompt.with_config({"tags": ["friendly", "helpful"]})
    | mistral_llm.with_config({"run_name": "Friendly_Mistral"})
    | StrOutputParser()
)
tagging_test = tagging_test.with_config({"run_name": "Friendly_Mistral"})
tagging_test.invoke(
    {
        "query": "Can you create a social media post for my blog about Artificial Intelligence and its impacts on our daily lives?"
    },
    config=RunnableConfig(metadata={"username": username, "session_id": session_id}),
)

'Absolutely, I\'d be happy to help you craft a social media post for your blog. Here\'s a suggestion:\n\n🔵🤖 New Blog Post Alert! 🤖🔵\n\n"Artificial Intelligence: Reshaping Our Daily Lives" is now live on our blog! 🚀\n\nHave you ever wondered how AI is subtly integrated into our everyday routines? From voice assistants like Siri and Alexa, to personalized recommendations on Netflix, AI is revolutionizing the way we live, work, and play. 🌐💻\n\nDive into our latest blog post to explore the fascinating world of AI and understand its profound impacts on our daily lives.\n\nLet\'s embrace the future, together! 🤝🌟\n\n#ArtificialIntelligence #AI #DailyLife #TechBlog #Innovation #FutureIsNow\n\nPlease feel free to modify it as per your preference.'

## LangSmith SDK and Tracer


In [30]:
from langchain_core.tracers.context import tracing_v2_enabled
from langsmith import Client

Client for interacting with the LangSmith API.


In [31]:
client = Client()
print(client.info, "\n")
print(client.read_project(project_name="barry-local-dev"), "\n")

version='0.2.21' license_expiration_time=None batch_ingest_config={'scale_up_qsize_trigger': 1000, 'scale_up_nthreads_limit': 16, 'scale_down_nempty_trigger': 4, 'size_limit': 100, 'size_limit_bytes': 20971520} 

id=UUID('d264aa8b-fbf9-44e4-af78-37edf5252812') start_time=datetime.datetime(2024, 2, 21, 5, 2, 29, 665079) end_time=None description=None name='barry-local-dev' extra=None tenant_id=UUID('d4f5e421-8f06-44c3-b1e0-8e69e2884cef') reference_dataset_id=None run_count=170 latency_p50=datetime.timedelta(seconds=5, microseconds=397000) latency_p99=datetime.timedelta(seconds=29, microseconds=629140) total_tokens=70392 prompt_tokens=33722 completion_tokens=36670 last_run_start_time=datetime.datetime(2024, 4, 12, 2, 25, 53, 814340) feedback_stats={} run_facets=[] 



Retrieve the tests results of a all runs a project as a panadas DataFrame object. Pandas needs to be installed.


In [32]:
df = client.get_test_results(project_name="barry-local-dev")
df.head()

,input.query,outputs.output,execution_time,error,id,input.Query,input.passage,input.location,input.unit,input.opposite,...,outputs.output.response_metadata.usage.input_tokens,outputs.output.response_metadata.usage.output_tokens,outputs.output.lat,outputs.output.lon,outputs.output.units,outputs.output.long,outputs.output.unit,outputs.output.city,outputs.output.state,outputs.output.country
0,Can you create a social media post for my blog...,"Absolutely, I'd be happy to help you craft a s...",7.120384,None,dee124c6-91c8-45f3-93ed-c06acbafdd92,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Can you create a social media post for my blog...,"Absolutely, I'd be happy to help you craft a s...",5.606131,None,bc3977a3-6aeb-4163-994e-768fb5e3d402,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Can you create a social media post for my blog...,Certainly! Here's a social media post that you...,13.466071,None,6ed0f806-9b7b-4a54-b052-54a40280274e,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Can you create a social media post for my blog...,**Headline:** Unlocking the Power of AI: How I...,4.176947,None,faf0c036-79a1-4045-81ca-54482e3f6cb2,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Whats the weather like on Chicago in Farenheit?,"**Personalized Weather Report**\n\nHey there, ...",4.745991,None,aba2c847-683b-4990-a5cc-75f764bd8c4c,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


With the tracer used in a context mannager you can leverage a call back to get information back about a run without logging to LangSmith.


In [33]:
with tracing_v2_enabled() as cb:
    prompt = ChatPromptTemplate.from_template(
        template="You are a helpful and harmless AI assistant. \n\nHuman: {query}"
    )
    basic_query_a = (
        prompt.with_config({"tags": ["Tracing Example"]})
        | anthropic_llm.with_config(
            {"run_name": "Anthropic-Example", "tags": ["Tracing Example"]}
        )
        | StrOutputParser().with_config({"tags": ["Tracing Example"]})
    )
    basic_query_a = basic_query_a.with_config({"run_name": "Tracing Example"})

    output = basic_query_a.invoke(
        {
            "query": "Can you create a social media post for my blog about Artificial Intelligence and its impacts on our daily lives?"
        }
    )
    print(f"Output: {output}\n\n")
    print(f"UUID FOR LATEST ID: {cb.latest_run.id}")
    print(f"URL LINK TO RUN {cb.latest_run.id}: {cb.get_run_url()}")

Output: Sure, I'd be happy to help you create a social media post for your blog about Artificial Intelligence (AI) and its impacts on our daily lives. Here's a sample post you can use or modify:

Title: The Impact of Artificial Intelligence on Our Daily Lives

Artificial Intelligence is transforming the way we live, work, and interact with the world around us. From virtual assistants like Siri and Alexa to personalized recommendations on streaming platforms, AI is becoming increasingly integrated into our daily routines.

But the impact of AI goes far beyond convenience and entertainment. In healthcare, AI is being used to analyze medical images, assist in diagnosis, and even develop new treatments. In education, AI-powered tools are providing personalized learning experiences and helping teachers to better understand student needs.

However, as with any new technology, there are also concerns about the potential downsides of AI. From job displacement to privacy risks, it's important t

Using the SDK to get the run information as a `Run` object.


In [34]:
output = client.read_run(cb.latest_run.id)
output

Run(id=UUID('e6e9188e-6974-44f9-b4f8-8524e813ffaf'), name='Tracing Example', start_time=datetime.datetime(2024, 4, 12, 2, 26, 25, 974846), run_type='chain', end_time=datetime.datetime(2024, 4, 12, 2, 26, 41, 244250), extra=None, error=None, serialized={'graph': {'edges': [{'source': 0, 'target': 1}, {'source': 1, 'target': 2}, {'source': 3, 'target': 4}, {'source': 2, 'target': 3}], 'nodes': [{'data': 'PromptInput', 'id': 0, 'type': 'schema'}, {'data': {'id': ['langchain', 'prompts', 'chat', 'ChatPromptTemplate'], 'name': 'ChatPromptTemplate'}, 'id': 1, 'type': 'runnable'}, {'data': {'id': ['langchain_anthropic', 'chat_models', 'ChatAnthropic'], 'name': 'ChatAnthropic'}, 'id': 2, 'type': 'runnable'}, {'data': {'id': ['langchain', 'schema', 'output_parser', 'StrOutputParser'], 'name': 'StrOutputParser'}, 'id': 3, 'type': 'runnable'}, {'data': 'StrOutputParserOutput', 'id': 4, 'type': 'schema'}]}, 'id': ['langchain', 'schema', 'runnable', 'RunnableSequence'], 'kwargs': {'first': {'graph'

### Evaluators


In [37]:
import langsmith
from langchain import smith
from langchain_openai import ChatOpenAI

# Replace with the chat model you want to test
my_llm = ChatGoogleGenerativeAI(google_api_key=google_api_key, model="gemini-pro")

# Define the evaluators to apply
eval_config = smith.RunEvalConfig(
    evaluators=["cot_qa"],
    custom_evaluators=[],
    eval_llm=ChatOpenAI(
        model="gpt-4-turbo", temperature=0, openai_api_key=openai_api_key
    ),
)

client = langsmith.Client()
chain_results = client.run_on_dataset(
    dataset_name="Weather API Generative Output",
    llm_or_chain_factory=my_llm,
    evaluation=eval_config,
    project_name=str(uuid4()),
    concurrency_level=5,
    verbose=True,
)

View the evaluation results for project '9c32d8c0-03fd-49ff-8561-296def7832e9' at:
https://smith.langchain.com/o/d4f5e421-8f06-44c3-b1e0-8e69e2884cef/datasets/b81aa58f-4785-4a3c-8ce4-81954a537065/compare?selectedSessions=5ef6a667-92ed-41c7-9351-e3f428ad7486

View all tests for Dataset Weather API Generative Output at:
https://smith.langchain.com/o/d4f5e421-8f06-44c3-b1e0-8e69e2884cef/datasets/b81aa58f-4785-4a3c-8ce4-81954a537065
[------------------------------------------------->] 13/13

,feedback.COT Contextual Accuracy,error,execution_time,run_id
count,13.000000,0,13.000000,13
unique,NaN,0,NaN,13
top,NaN,NaN,NaN,676c7a67-c8d5-428b-8156-bbd4968c0cd8
freq,NaN,NaN,NaN,1
mean,0.076923,NaN,3.364477,NaN
std,0.277350,NaN,0.551714,NaN
min,0.000000,NaN,2.573565,NaN
25%,0.000000,NaN,2.961449,NaN
50%,0.000000,NaN,3.279264,NaN
75%,0.000000,NaN,3.888144,NaN
